In [47]:
import openai
import pandas as pd

gpt설정

In [48]:
def question(query):

    # FTA API
    API_KEY = ""
    openai.api_key = API_KEY

    model = "gpt-3.5-turbo"  

    content = """
You are a binary classifier.
When programming code comes in, you perform binary classification according to the given rules.
Classify as 1 if it meets the rule, otherwise classify as 0.
You can only answer about binary classification results. The results are in the form of a list.
"""

    messages = [
        {"role":"system", "content": content},
        {"role":"user","content":query}
    ]

    response = openai.ChatCompletion.create(
        temperature=0,
        model=model,
        messages = messages
    )

    answer = response['choices'][0]['message']['content']

    return answer


예측 결과 컬럼 생성

In [49]:
df = pd.read_excel("../문제별정답코드.xlsx") 
df['prediction1'] = None
df['prediction2'] = None
df['prediction3'] = None
df['prediction4'] = None
df.head()

,문제 이름,정답 코드,multiple input,multiple output,conditional statement,mathematical calculation,prediction1,prediction2,prediction3,prediction4
0,10보다작을까?-1,"#include<cstdio>\nint main(){\nint a;\nscanf(""%d"", &a);\nif (a<10){\nprintf(""hello"");\n}\n}",0,0,1,0,None,None,None,None
1,10보다작을까?-2,"#include <cstdio>\nint main(){\n int a;\n scanf(""%d"",&a);\n if(a<10)\n {\n printf(""hello"");\n }\n else{\n printf(""algo"");\n }\n \n}",0,0,1,0,None,None,None,None
2,내나이가어때서,"#include <cstdio>\nint main()\n{\n int a,b;\n scanf(""%d %d"",&a,&b);\n printf(""%d"",a-b+1);\n return 0;\n}",1,0,0,1,None,None,None,None
3,두수비교하기(<=),"#include <cstdio>\nint main()\n{\n int a,b;\n scanf(""%d %d"",&a,&b);\n if(a<=b){\n printf("":("");\n }\n else\n {\n printf("":)"");\n }\n}",1,0,1,0,None,None,None,None
4,두수비교하기(==),"#include <cstdio>\nint main()\n{\n int a,b;\n scanf(""%d %d"",&a,&b);\n if(a==b)\n {\n printf("":)"");\n }\n else\n {\n printf("":("");\n}\n}",1,0,1,0,None,None,None,None


query에 넣을 df가공

In [50]:
pd.set_option('display.max_colwidth', None)
query_df = df["정답 코드"]
query_df

query_df.head()

0                                                                                        #include<cstdio>\nint main(){\nint a;\nscanf("%d", &a);\nif (a<10){\nprintf("hello");\n}\n}
1    #include <cstdio>\nint main(){\n    int a;\n    scanf("%d",&a);\n    if(a<10)\n    {\n        printf("hello");\n    }\n    else{\n        printf("algo");\n    }\n          \n}
2                                                               #include <cstdio>\nint main()\n{\n    int a,b;\n    scanf("%d %d",&a,&b);\n    printf("%d",a-b+1);\n    return 0;\n}
3           #include <cstdio>\nint main()\n{\n    int a,b;\n    scanf("%d %d",&a,&b);\n    if(a<=b){\n        printf(":(");\n    }\n    else\n    {\n        printf(":)");\n    }\n}
4         #include <cstdio>\nint main()\n{\n    int a,b;\n    scanf("%d %d",&a,&b);\n    if(a==b)\n    {\n        printf(":)");\n    }\n    else\n    {\n        printf(":(");\n}\n}
Name: 정답 코드, dtype: object

query 함수 작성

In [65]:
def code_binary_classifier(rule, df_copy):
    size = len(df_copy)

    except_query = ""
    for _ in range(3):
        question_query = f"""
1. Question :
    The following dataframe represents the solution codes for each question.

    Your task is to classify the solution code for each question into binary categories according to the rules below:

"""

        result_query = """
2. Answer : 
    you only need to answer the classification results, and the format of the results is as shown in the list below. The list index matches each solution code.
    Example:
    [0, 1, ... ]

"""

        context_query = f"The DataFrame contains the following {size} codes.\n\n"  + df_copy.to_string()

        query = except_query + question_query + rule + result_query + context_query

        ans = question(query)
        

        try:
            result = eval(ans)

            if len(result) == size:
                break
            else:
                except_query = f"Previously, you provided {len(result)} results, but the dataframe has {size}. Please answer accurately again."
                print("The size of the result and DataFrame do not match. Retrying...")
        except:
            print("An error occurred during execution. Retrying...")

    return result



multple input 예측

In [73]:
rule1 = """
rule:
If there are multiple input variables, classify as 1, otherwise classify as 0.

Example:
scanf("%d %d", &a, &b) -> 1 #2 input variables
scanf("%d %d %d", &n, &p, &q) -> 1 #3 input variables
scanf("%d", &a) -> 0 #1 input variables
scanf("%lf, &n) -> 0 #1 input variables

"""

df.loc[ :, "prediction1"] = code_binary_classifier(rule1, query_df)

df.loc[:, ["문제 이름", "multiple input", "prediction1"]]

,문제 이름,multiple input,prediction1
0,10보다작을까?-1,0,0
1,10보다작을까?-2,0,0
2,내나이가어때서,1,1
3,두수비교하기(<=),1,1
4,두수비교하기(==),1,1
5,두정수바꾸어출력하기,1,1
6,두정수입력받기,1,1
7,반값이벤트,1,1
8,사칙연산,1,1
9,시간단위로변환하기,0,0


multiple output 예측

In [74]:
rule2 = """
rule:
If there are multiple output variables, classify as 1, otherwise classify as 0.

Example:
1. printf("jang") -> 0 #Output a single string
2. printf("%d", a) -> 0 #Output 1 variable.
3. printf("%lf", a+b+5) -> 0 #Output 1 variable.
4. if (a>1){ printf("%d", &a) } else{ printf("sdf") } -> 0 #Output 1 variable.
5. printf("%d %d", a,b) -> 1 #Output 2 variable.
6. printf("%d %lf", q,r) -> 1 #Output 2 variable.
7. printf("%d %lf %d", q,r, q*r+2) -> 1 #Output 3 variable.

"""

df.loc[ :, "prediction2"] = code_binary_classifier(rule2, query_df)

df.loc[:, ["문제 이름", "multiple output", "prediction2"]]

,문제 이름,multiple output,prediction2
0,10보다작을까?-1,0,0
1,10보다작을까?-2,0,0
2,내나이가어때서,0,0
3,두수비교하기(<=),0,0
4,두수비교하기(==),0,0
5,두정수바꾸어출력하기,1,1
6,두정수입력받기,1,1
7,반값이벤트,0,0
8,사칙연산,1,1
9,시간단위로변환하기,0,0


conditional statement 예측

In [75]:
rule3 = """
rule:
If there are conditional(if) statements in each code, classify as 1, otherwise classify as 0.

Example:
1. |
#include <cstdio>

int main()
{
int a;
scanf("%d", &a);

if (a < 10)
{
printf("hello");
}

return 0;
}
| -> 1 #If statement exists

2. |
#include <cstdio>

int main()
{
    int a,b;
    scanf("%d %d",&a,&b);
    printf("%d %d",a,b);
    
    return 0;
}
| -> 0 #If statement does not exist

"""

df.loc[ :, "prediction3"] = code_binary_classifier(rule3, query_df)

df.loc[:, ["문제 이름", "conditional statement", "prediction3"]]

,문제 이름,conditional statement,prediction3
0,10보다작을까?-1,1,1
1,10보다작을까?-2,1,1
2,내나이가어때서,0,0
3,두수비교하기(<=),1,1
4,두수비교하기(==),1,1
5,두정수바꾸어출력하기,0,0
6,두정수입력받기,0,0
7,반값이벤트,0,0
8,사칙연산,0,0
9,시간단위로변환하기,0,0


mathematical calculation 예측

In [76]:
rule4 = """
rule:
If there exist arithmetic operations (+, -, *, /, %), classify as 1 otherwise classify as 0.

Example:
1. printf("%d",a*8); | -> 1
2. printf("%d %d",a ,a*b); | -> 1
3. q = a + 5; | -> 1


"""

df.loc[ :, "prediction4"] = code_binary_classifier(rule4, query_df)

df.loc[:, ["문제 이름", "mathematical calculation", "prediction4"]]

,문제 이름,mathematical calculation,prediction4
0,10보다작을까?-1,0,0
1,10보다작을까?-2,0,0
2,내나이가어때서,1,1
3,두수비교하기(<=),0,1
4,두수비교하기(==),0,1
5,두정수바꾸어출력하기,0,0
6,두정수입력받기,0,0
7,반값이벤트,1,1
8,사칙연산,1,1
9,시간단위로변환하기,1,1


정확도 평가

In [77]:
df_copy = df
df_copy = df_copy.drop(["문제 이름", "정답 코드"], axis=1)

print(df_copy.to_string(index=False))

 multiple input  multiple output  conditional statement  mathematical calculation prediction1 prediction2 prediction3 prediction4
              0                0                      1                         0           0           0           1           0
              0                0                      1                         0           0           0           1           0
              1                0                      0                         1           1           0           0           1
              1                0                      1                         0           1           0           1           1
              1                0                      1                         0           1           0           1           1
              1                1                      0                         0           1           1           0           0
              1                1                      0                         0         

In [78]:
# 정확도 평가 함수
def calculate_accuracy(predictions, targets):
    return (predictions == targets).mean()

In [79]:
# 각 항목에 대한 정확도 평가
accuracy1 = calculate_accuracy(df['prediction1'], df['multiple input'])
accuracy2 = calculate_accuracy(df['prediction2'], df['multiple output'])
accuracy3 = calculate_accuracy(df['prediction3'], df['conditional statement'])
accuracy4 = calculate_accuracy(df['prediction4'], df['mathematical calculation'])

print(f"Accuracy for multiple input: {accuracy1:.2f}")
print(f"Accuracy for multiple output: {accuracy2:.2f}")
print(f"Accuracy for conditional statement: {accuracy3:.2f}")
print(f"Accuracy for mathematical calculation: {accuracy4:.2f}")

Accuracy for multiple input: 0.86
Accuracy for multiple output: 0.86
Accuracy for conditional statement: 1.00
Accuracy for mathematical calculation: 0.79


In [80]:
# 정확도 평균 계산
average_accuracy = (accuracy1 + accuracy2 + accuracy3 + accuracy4) / 4

print(f"Average Accuracy: {average_accuracy:.2f}")

Average Accuracy: 0.88


In [28]:
df.to_excel('../문제별정답코드(예측포함).xlsx', index=False, engine='openpyxl')